<a href="https://colab.research.google.com/github/singhvarun404/AIML_2020/blob/master/August_20th_Task_2_Breast_Cancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import tensorflow.keras as tf
import numpy as np
import pandas as pd

In [2]:
cancer_df = pd.read_csv('breast_cancer.csv')
cancer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

In [3]:
cancer_df['diagnosis'].value_counts()

B    357
M    212
Name: diagnosis, dtype: int64

In [4]:
### Data visualization
###

In [5]:
x_feat = cancer_df.iloc[:,2:32].values
x_feat.shape

(569, 30)

In [6]:
y_lab = cancer_df.iloc[:,1].values
y_lab.shape

(569,)

In [7]:
for i in range(len(y_lab)):
  if y_lab[i] == "B":
    y_lab[i] = 0
  else:
    y_lab[i] = 1

In [8]:
y_lab.dtype

dtype('O')

In [9]:
y_lab = y_lab.astype(np.float32)

In [10]:
y_lab.dtype

dtype('float32')

In [11]:
### Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_feat = sc.fit_transform(x_feat)

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
### Ensuring distribution of malignant and benign data is uniform among test and train set
rs = 1
while(1):
  x_feat_train, x_feat_test, y_lab_train, y_lab_test = train_test_split(x_feat, y_lab, test_size = 0.2, random_state = rs)
  no_of_M = 0
  no_of_B = 0
  for i in range(len(y_lab_test)):
    if y_lab_test[i] == 0:
      no_of_B += 1
    else:
      no_of_M += 1
  ratio_BM = no_of_B/no_of_M
  if ratio_BM <= 1.7 and ratio_BM >=1.6:
    break
  else:
    rs += 1
    continue
rs

22

In [14]:
### Feature Scaling
#from sklearn.preprocessing import StandardScaler
#sc = StandardScaler()
#x_feat_train = sc.fit_transform(x_feat_train)
#X_feat_test = sc.transform(x_feat_test)

In [15]:
### Creating the model and training
from tensorflow.keras import regularizers
model = tf.models.Sequential()

model.add(tf.layers.Dense(30,activation = 'relu',kernel_regularizer=regularizers.l2(0.002),input_dim = 30))   
model.add(tf.layers.Dense(60,activation = 'relu',kernel_regularizer=regularizers.l2(0.002))) 
model.add(tf.layers.Dense(1,activation = 'sigmoid')) 

optimiz = tf.optimizers.Adam(learning_rate=0.0005)

model.compile(loss = 'binary_crossentropy', optimizer = optimiz, metrics = ['accuracy'])

In [16]:
model.fit(x_feat_train,y_lab_train,batch_size = 5,epochs=20)

Epoch 1/20
91/91 [==============================] - 0s 1ms/step - loss: 0.5520 - accuracy: 0.8571
Epoch 2/20
91/91 [==============================] - 0s 1ms/step - loss: 0.3188 - accuracy: 0.9538
Epoch 3/20
91/91 [==============================] - 0s 1ms/step - loss: 0.2515 - accuracy: 0.9648
Epoch 4/20
91/91 [==============================] - 0s 1ms/step - loss: 0.2206 - accuracy: 0.9670
Epoch 5/20
91/91 [==============================] - 0s 1ms/step - loss: 0.2003 - accuracy: 0.9780
Epoch 6/20
91/91 [==============================] - 0s 1ms/step - loss: 0.1849 - accuracy: 0.9868
Epoch 7/20
91/91 [==============================] - 0s 1ms/step - loss: 0.1728 - accuracy: 0.9890
Epoch 8/20
91/91 [==============================] - 0s 1ms/step - loss: 0.1632 - accuracy: 0.9912
Epoch 9/20
91/91 [==============================] - 0s 1ms/step - loss: 0.1543 - accuracy: 0.9912
Epoch 10/20
91/91 [==============================] - 0s 1ms/step - loss: 0.1469 - accuracy: 0.9912
Epoch 11/20
91/91 [

In [17]:
### Testing
y_pred = model.predict(x_feat_test)
y_prede = []
for i in range(len(y_pred)):
  if y_pred[i] < 0.5:
    y_prede.append(0.0)
  else:
    y_prede.append(1.0)
y_pred_npa = np.array(y_prede)

In [18]:
y_pred_npa[10]

0.0

In [19]:
from tensorflow.math import confusion_matrix
conf = confusion_matrix(y_lab_test,y_pred_npa,num_classes=2)
conf

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[70,  1],
       [ 3, 40]], dtype=int32)>

In [20]:
from tensorflow.keras.metrics import binary_accuracy

In [21]:
acc = binary_accuracy(y_lab_test,y_pred_npa)
acc

<tf.Tensor: shape=(), dtype=float32, numpy=0.9649123>